<a href="https://colab.research.google.com/github/FatemeZamanian/HousePrices/blob/main/HousePrices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
!pip install Wandb

In [108]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [225]:
import wandb
wandb.init(project='HousePrices', entity='fatemezamanian')

epoch,199
loss,0.16051
val_loss,0.15968
_runtime,417
_timestamp,1631536413
_step,199
best_val_loss,0.15968
best_epoch,199


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


In [209]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [210]:
data=pd.read_csv('/content/drive/MyDrive/train_HousePrices.csv')
#street
data=data.replace(['Pave','Grvl'],[0,1])
#MsZoning
data=data.replace(['A','C (all)','FV','I','RH','RL','RP','RM'],[0,1,2,3,4,5,6,7])
#lotarea
#alley
data=data.replace(['Grvl','Pave','NA'],[0,1,2])
#landcomntour
data=data.replace(['Lvl','Bnk','HLS','Low'],[0,1,2,3])
#utilities
data=data.replace(['AllPub','NoSewr','NoSeWa','ELO'],[0,1,2,3])
#BldgType
data=data.replace(['1Fam','2fmCon','Duplex','TwnhsE','Twnhs'],[0,1,2,3,4])
#OverallQual
#OverallCond
#BsmtCond
data=data.replace(['Ex','Gd','TA','Fa','Po','NA'],[0,1,2,3,4,5])
#BsmtFinType1
data=data.replace(['GLQ','ALQ','BLQ','Rec','LwQ','Unf','NA'],[0,1,2,3,4,5,6])
#TotalBsmtSF
#HeatingQC: Heating quality and condition
data=data.replace(['Ex','Gd','TA','Fa','Po'],[0,1,2,3,4])
#CentralAir: Central air conditioning
data=data.replace(['N','Y'],[0,1])
#GrLivArea
#KitchenQual
#TotRmsAbvGrd
#GarageCond: Garage condition
#PoolQC
#YearBuilt
#MiscVal: $Value of miscellaneous feature
data=data.fillna(0)
data.head()



,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,5,65.0,8450,0,0.0,Reg,0,0,Inside,Gtl,CollgCr,Norm,Norm,0,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,1,2,PConc,1.0,2.0,No,0.0,706,5.0,0,150,856,GasA,...,1,SBrkr,856,854,0,1710,1,0,2,1,3,1,1,8,Typ,0,0.0,Attchd,2003.0,RFn,2,548,2.0,2.0,1,0,61,0,0,0,0,0.0,0,0,0,2,2008,WD,Normal,208500
1,2,20,5,80.0,9600,0,0.0,Reg,0,0,FR2,Gtl,Veenker,Feedr,Norm,0,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,2,2,CBlock,1.0,2.0,1,1.0,978,5.0,0,284,1262,GasA,...,1,SBrkr,1262,0,0,1262,0,1,2,0,3,1,2,6,Typ,1,2.0,Attchd,1976.0,RFn,2,460,2.0,2.0,1,298,0,0,0,0,0,0.0,0,0,0,5,2007,WD,Normal,181500
2,3,60,5,68.0,11250,0,0.0,IR1,0,0,Inside,Gtl,CollgCr,Norm,Norm,0,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,1,2,PConc,1.0,2.0,Mn,0.0,486,5.0,0,434,920,GasA,...,1,SBrkr,920,866,0,1786,1,0,2,1,3,1,1,6,Typ,1,2.0,Attchd,2001.0,RFn,2,608,2.0,2.0,1,0,42,0,0,0,0,0.0,0,0,0,9,2008,WD,Normal,223500
3,4,70,5,60.0,9550,0,0.0,IR1,0,0,Corner,Gtl,Crawfor,Norm,Norm,0,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,2,2,BrkTil,2.0,1.0,No,1.0,216,5.0,0,540,756,GasA,...,1,SBrkr,961,756,0,1717,1,0,1,0,3,1,1,7,Typ,1,1.0,Detchd,1998.0,5,3,642,2.0,2.0,1,0,35,272,0,0,0,0.0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,5,84.0,14260,0,0.0,IR1,0,0,FR2,Gtl,NoRidge,Norm,Norm,0,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,1,2,PConc,1.0,2.0,Av,0.0,655,5.0,0,490,1145,GasA,...,1,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,1,9,Typ,1,2.0,Attchd,2000.0,RFn,3,836,2.0,2.0,1,192,84,0,0,0,0,0.0,0,0,0,12,2008,WD,Normal,250000


In [219]:
X=data[['Street','MSZoning','LotArea','Alley','LandContour','Utilities','BldgType','OverallQual','OverallCond','BsmtCond','BsmtFinType1','TotalBsmtSF','HeatingQC','CentralAir',
   'GrLivArea','KitchenQual', 'TotRmsAbvGrd','GarageCond','PoolQC','YearBuilt','MiscVal']]
Y=data[['SalePrice']]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=25)
X_train=np.array(X_train) 
X_test=np.array(X_test) 
Y_train=np.array(Y_train) 
Y_test=np.array(Y_test)
#X.head(50)

In [226]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(17, activation='relu'),
    tf.keras.layers.Dense(289, activation='relu'),
    tf.keras.layers.Dense(8670, activation='relu'),
    tf.keras.layers.Dense(145, activation='relu'),
    tf.keras.layers.Dense(34, activation='relu'),
    tf.keras.layers.Dense(17, activation='relu'),
    tf.keras.layers.Dense(1, activation='relu')
])

In [227]:
from wandb.keras import WandbCallback
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.009),
              loss=tf.keras.losses.mean_squared_logarithmic_error)

In [228]:
config = wandb.config
config.learning_rate = 0.009

In [229]:
out=model.fit(X_train,Y_train,epochs=200,validation_data=(X_test,Y_test),callbacks=[WandbCallback()])

Epoch 1/200
39/39 [==============================] - 2s 43ms/step - loss: 1.6116 - val_loss: 0.1758
Epoch 2/200
39/39 [==============================] - 2s 40ms/step - loss: 0.1771 - val_loss: 0.1699
Epoch 3/200
39/39 [==============================] - 1s 38ms/step - loss: 0.1709 - val_loss: 0.1652
Epoch 4/200
39/39 [==============================] - 2s 39ms/step - loss: 0.1653 - val_loss: 0.1612
Epoch 5/200
39/39 [==============================] - 1s 37ms/step - loss: 0.1597 - val_loss: 0.1565
Epoch 6/200
39/39 [==============================] - 2s 39ms/step - loss: 0.1544 - val_loss: 0.1526
Epoch 7/200
39/39 [==============================] - 2s 39ms/step - loss: 0.1491 - val_loss: 0.1477
Epoch 8/200
39/39 [==============================] - 2s 39ms/step - loss: 0.1437 - val_loss: 0.1421
Epoch 9/200
39/39 [==============================] - 2s 39ms/step - loss: 0.1387 - val_loss: 0.1374
Epoch 10/200
39/39 [==============================] - 1s 38ms/step - loss: 0.1339 - val_loss: 0.1338

In [241]:
test_data=pd.read_csv('/content/drive/MyDrive/test_HousePrices.csv')
test_data=test_data.replace(['Pave','Grvl'],[0,1])
test_data=test_data.replace(['A','C (all)','FV','I','RH','RL','RP','RM'],[0,1,2,3,4,5,6,7])
test_data=test_data.replace(['Grvl','Pave','NA'],[0,1,2])
test_data=test_data.replace(['Lvl','Bnk','HLS','Low'],[0,1,2,3])
test_data=test_data.replace(['AllPub','NoSewr','NoSeWa','ELO'],[0,1,2,3])
test_data=test_data.replace(['1Fam','2fmCon','Duplex','TwnhsE','Twnhs'],[0,1,2,3,4])
test_data=test_data.replace(['Ex','Gd','TA','Fa','Po','NA'],[0,1,2,3,4,5])
test_data=test_data.replace(['GLQ','ALQ','BLQ','Rec','LwQ','Unf','NA'],[0,1,2,3,4,5,6])
test_data=test_data.replace(['Ex','Gd','TA','Fa','Po'],[0,1,2,3,4])
test_data=test_data.replace(['N','Y'],[0,1])
test_data=test_data.fillna(0)
test_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,4.0,80.0,11622,0,0.0,Reg,0,0.0,Inside,Gtl,NAmes,Feedr,Norm,0,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,2,2,CBlock,2.0,2.0,No,3.0,468.0,4.0,144.0,270.0,882.0,GasA,2,1,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,2.0,5,Typ,0,0.0,Attchd,1961.0,5,1.0,730.0,2.0,2.0,1,140,0,0,0,120,0,0.0,MnPrv,0,0,6,2010,WD,Normal
1,1462,20,5.0,81.0,14267,0,0.0,IR1,0,0.0,Corner,Gtl,NAmes,Norm,Norm,0,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,2,2,CBlock,2.0,2.0,No,1.0,923.0,5.0,0.0,406.0,1329.0,GasA,2,1,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,1.0,6,Typ,0,0.0,Attchd,1958.0,5,1.0,312.0,2.0,2.0,1,393,36,0,0,0,0,0.0,0,Gar2,12500,6,2010,WD,Normal
2,1463,60,5.0,74.0,13830,0,0.0,IR1,0,0.0,Inside,Gtl,Gilbert,Norm,Norm,0,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,2,2,PConc,1.0,2.0,No,0.0,791.0,5.0,0.0,137.0,928.0,GasA,1,1,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,2.0,6,Typ,1,2.0,Attchd,1997.0,Fin,2.0,482.0,2.0,2.0,1,212,34,0,0,0,0,0.0,MnPrv,0,0,3,2010,WD,Normal
3,1464,60,5.0,78.0,9978,0,0.0,IR1,0,0.0,Inside,Gtl,Gilbert,Norm,Norm,0,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,2,2,PConc,2.0,2.0,No,0.0,602.0,5.0,0.0,324.0,926.0,GasA,0,1,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,1.0,7,Typ,1,1.0,Attchd,1998.0,Fin,2.0,470.0,2.0,2.0,1,360,36,0,0,0,0,0.0,0,0,0,6,2010,WD,Normal
4,1465,120,5.0,43.0,5005,0,0.0,IR1,2,0.0,Inside,Gtl,StoneBr,Norm,Norm,3,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,1,2,PConc,1.0,2.0,No,1.0,263.0,5.0,0.0,1017.0,1280.0,GasA,0,1,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,1.0,5,Typ,0,0.0,Attchd,1992.0,RFn,2.0,506.0,2.0,2.0,1,0,82,0,0,144,0,0.0,0,0,0,1,2010,WD,Normal


In [243]:
X_t=test_data[['Street','MSZoning','LotArea','Alley','LandContour','Utilities','BldgType','OverallQual','OverallCond','BsmtCond','BsmtFinType1','TotalBsmtSF','HeatingQC','CentralAir',
   'GrLivArea','KitchenQual', 'TotRmsAbvGrd','GarageCond','PoolQC','YearBuilt','MiscVal']]
X_t=np.array(X_t)
# X_t.head()

In [244]:
Y_t=pd.read_csv('/content/drive/MyDrive/y_test_HousePrices.csv')
Y_t = Y_t[['SalePrice']]
Y_t=np.array(Y_t)

In [245]:
model.evaluate(X_t,Y_t)

46/46 [==============================] - 0s 10ms/step - loss: 0.0854


0.08535271883010864